In [161]:
import os
import win32com.client as win32
import numpy as np

In [162]:
class Simulation:
    AspenSimulation = win32.gencache.EnsureDispatch("Apwn.Document")

    def __init__(self, AspenFileName, WorkingDirectoryPath, VISIBILITY=False):
        os.chdir(WorkingDirectoryPath)
        print(f"Working Directory: {os.getcwd()}")
        print(f"Aspen File: {AspenFileName}")
        self.AspenSimulation.InitFromArchive2(os.path.abspath(AspenFileName))
        self.AspenSimulation.Visible = VISIBILITY
        self.AspenSimulation.SuppressDialogs = True

    def CloseAspen(self):
        AspenFileName = self.Give_AspenDocumentName()
        self.AspenSimulation.Close(os.path.abspath(AspenFileName))
        
    def Quit(self):
        self.AspenSimulation.Quit()
        
    def Give_AspenDocumentName(self):
        return self.AspenSimulation.FullName

    @property
    def BLK(self):
        return self.AspenSimulation.Tree.Elements("Data").Elements("Blocks")

    @property
    def STRM(self):
        return self.AspenSimulation.Tree.Elements("Data").Elements("Streams")

    def EngineRun(self):
        self.AspenSimulation.Run2()

    def EngineStop(self):
        self.AspenSimulation.Stop()

    def EngineReinit(self):
        self.AspenSimulation.Reinit()

    def Convergence(self):
        converged = (
            self.AspenSimulation.Tree.Elements("Data")
            .Elements("Results Summary")
            .Elements("Run-Status")
            .Elements("Output")
            .Elements("PER_ERROR")
            .Value
        )
        return converged == 0

    def StreamConnect(self, Blockname, Streamname, Portname):
        self.BLK.Elements(Blockname).Elements("Ports").Elements(Portname).Elements.Add(
            Streamname
        )

    def StreamDisconnect(self, Blockname, Streamname, Portname):
        self.BLK.Elements(Blockname).Elements("Ports").Elements(
            Portname
        ).Elements.Remove(Streamname)

    def Reinitialize(self):
        self.STRM.RemoveAll()
        self.BLK.RemoveAll()
        self.AspenSimulation.Reinit()


class Stream(Simulation):
    def __init__(self, name, inlet=False):
        self.name = name.upper()
        self.inlet = inlet

        self.StreamPlace()

        if self.inlet:
            self.inlet_stream()

    def StreamPlace(self):
        compositstring = self.name + "!" + "MATERIAL"
        self.STRM.Elements.Add(compositstring)

    def StreamDelete(self):
        self.STRM.Elements.Remove(self.name)

    def inlet_stream(self):
        T = self.inlet[0]
        P = self.inlet[1]
        comp = self.inlet[2]

        self.STRM.Elements(self.name).Elements("Input").Elements("TEMP").Elements(
            "MIXED"
        ).Value = T
        self.STRM.Elements(self.name).Elements("Input").Elements("PRES").Elements(
            "MIXED"
        ).Value = P

        for chemical in comp:
            self.STRM.Elements(self.name).Elements("Input").Elements("FLOW").Elements(
                "MIXED"
            ).Elements(chemical).Value = comp[chemical]

    def get_temp(self):
        return (
            self.STRM.Elements(self.name)
            .Elements("Output")
            .Elements("TEMP_OUT")
            .Elements("MIXED")
            .Value
        )

    def get_press(self):
        return (
            self.STRM.Elements(self.name)
            .Elements("Output")
            .Elements("PRES_OUT")
            .Elements("MIXED")
            .Value
        )

    def get_molar_flow(self, compound):
        return (
            self.STRM.Elements(self.name)
            .Elements("Output")
            .Elements("MOLEFLOW")
            .Elements("MIXED")
            .Elements(compound)
            .Value
        )

    def get_total_molar_flow(self):
        return (
            self.STRM.Elements(self.name)
            .Elements("Output")
            .Elements("MOLEFLMX")
            .Elements("MIXED")
            .Value
        )

    def get_vapor_fraction(self):
        return (
            self.STRM.Elements(self.name)
            .Elements("Output")
            .Elements("STR_MAIN")
            .Elements("VFRAC")
            .Elements("MIXED")
            .Value
        )
    
    def get_mass_flow(self):
        return (
            self.STRM.Elements(self.name)
            .Elements("Output")
            .Elements("MASSFLMX")
            .Elements("MIXED")
            .Value
        )
    def stream_specs(self):
        return (self.get_temp(), self.get_press(), self.get_mass_flow())


class Block(Simulation):
    def __init__(self, name, uo):
        self.name = name.upper()
        self.uo = uo

    def BlockCreate(self):
        compositestring = self.name + "!" + self.uo
        self.BLK.Elements.Add(compositestring)

    def BlockDelete(self):
        self.BLK.Elements.Remove(self.name)
        

In [163]:
# -------------------------------------------------- UNIT OPERATIONS ------------------------------------------------
class InitialMixer(Block):
    def __init__(self, name, inlet_streams):
        super().__init__(name, "Mixer")
        self.name = name
        self.inlet_streams = inlet_streams
        self.BlockCreate()

    def connect(self):
        # Inlet connection
        for stream in self.inlet_streams:
            self.StreamConnect(self.name, stream.name, "F(IN)")

        # self.BLK.Elements(self.name).Elements("Input").Elements("NPHASE").Value = 1
        # self.BLK.Elements(self.name).Elements("Input").Elements("PHASE").Value = "L"

        s = Stream(f"{self.name}OUT")
        self.StreamConnect(self.name, s.name, "P(OUT)")
        return s
    def dv_placement(self, *args):
        # This method can be used to set any design variable for the mixer
        # Currently, no specific design variables are set for the mixer
        pass

class Mixer(Block):
    def __init__(self, name,inlet_streams):
        super().__init__(name, "Mixer")
        self.name = name
        self.inlet_streams = inlet_streams
        self.BlockCreate()

    def connect(self,stream):
        self.StreamConnect(self.name, stream.name, "F(IN)")
        s = Stream(f"{self.name}OUT")
        self.StreamConnect(self.name, s.name, "P(OUT)")
        return s
    def dv_placement(self, *args):
        # This method can be used to set any design variable for the mixer
        # Currently, no specific design variables are set for the mixer
        pass

class Splitter(Block):
    def __init__(self, name, inlet_stream):
        super().__init__(name, "FSplit")

        self.name = name
        self.inlet_stream = inlet_stream
        self.sr = 0.5  # Default split ratio
        self.BlockCreate()

    def connect(self):
        self.StreamConnect(self.name, self.inlet_stream.name, "F(IN)")

        self.s1 = Stream(f"{self.name}S1OUT")
        s2 = Stream(f"{self.name}S2OUT")

        self.StreamConnect(self.name, self.s1.name, "P(OUT)")
        self.StreamConnect(self.name, s2.name, "P(OUT)")

        return self.s1, s2
    
    def dv_placement(self, split_ratio):
        self.BLK.Elements(self.name).Elements("Input").Elements("FRAC").Elements(
            self.s1.name
        ).Value = split_ratio

class Heater(Block):
    def __init__(self, name,inlet_stream):
        super().__init__(name, "Heater")

        self.name = name
        self.inlet_stream = inlet_stream
        self.Temp = 298.15  # Default temperature in Kelvin
        self.BlockCreate()

    def connect(self):
        self.StreamConnect(self.name, self.inlet_stream.name, "F(IN)")

        self.BLK.Elements(self.name).Elements("Input").Elements("SPEC_OPT").Value = "TDPPARM"
        self.BLK.Elements(self.name).Elements("Input").Elements("DPPARM").Value = "0"
        s = Stream(f"{self.name}OUT")
        self.StreamConnect(self.name, s.name, "P(OUT)")
        return s
    
    def dv_placement(self, temp):
        self.BLK.Elements(self.name).Elements("Input").Elements("TEMP").Value = temp

    def enery_consumption(self):
        q = abs(self.BLK.Elements(self.name).Elements("Output").Elements("QCALC").Value)
        return q

class Cooler(Block):
    def __init__(self, name, inlet_stream):
        super().__init__(name, "Heater")

        self.name = name 
        self.inlet_stream = inlet_stream
        self.Temp = 298.15 
        self.BlockCreate()

    def connect(self):
        self.StreamConnect(self.name, self.inlet_stream.name, "F(IN)")

        self.BLK.Elements(self.name).Elements("Input").Elements("SPEC_OPT").Value = "TDPPARM"
        self.BLK.Elements(self.name).Elements("Input").Elements("DPPARM").Value = "0"
        s = Stream(f"{self.name}OUT")
        self.StreamConnect(self.name, s.name, "P(OUT)")
        return s
    
    def dv_placement(self, temp):
        self.BLK.Elements(self.name).Elements("Input").Elements("TEMP").Value = temp

    def enery_consumption(self):
        q = abs(self.BLK.Elements(self.name).Elements("Output").Elements("QCALC").Value)
        return q

class HeatExchanger(Block):
    def __init__(self, name,inlet_stream1, inlet_stream2=None):
        super().__init__(name, "HeatX")
        self.name = name
        self.inlet_stream1 = inlet_stream1
        self.inlet_stream2 = inlet_stream2
        self.BlockCreate()
           # self.hotside_pres = pres
            # self.BLK.Elements(self.name).Elements("Input").Elements(
            # "PRES_HOT").Value = self.hotside_pres
            #             self.coldside_pres = pres
            # self.BLK.Elements(self.name).Elements("Input").Elements(
            # "PRES_COLD").Value = self.coldside_pres

    def connect(self,hex_count,sin2=None):
        if hex_count == 1:
            self.StreamConnect(self.name, self.inlet_stream1.name, "H(IN)")
            s = Stream(f"{self.name}OUT1")
            self.StreamConnect(self.name, s.name, "H(OUT)")
            self.BLK.Elements(self.name).Elements("Input").Elements(
                "SPEC"
            ).Value = "DELT-HOT"
            self.outlet1 = s
        else:
            self.inlet_stream2 = sin2
            self.StreamConnect(self.name, self.inlet_stream2.name, "C(IN)")
            s = Stream(f"{self.name}OUT2")
            self.StreamConnect(self.name, s.name, "C(OUT)")
            self.outlet2 = s
        return s
    
    def dv_placement(self, DT):
            self.BLK.Elements(self.name).Elements("Input").Elements(
                "VALUE"
            ).Value = DT
    def switch_streams(self):
        # Switch the inlet streams for the heat exchanger
        self.StreamDisconnect(self.name, self.inlet_stream1.name, "H(IN)")
        self.StreamDisconnect(self.name, self.inlet_stream2.name, "C(IN)")
        self.StreamConnect(self.name, self.inlet_stream2.name, "H(IN)")
        self.StreamConnect(self.name, self.inlet_stream1.name, "C(IN)")
        new_coldside_pres = self.hotside_pres
        new_hotside_pres = self.coldside_pres        

        # Update the outlet streams accordingly
        self.StreamDisconnect(self.name, self.outlet1.name, "H(OUT)")
        self.StreamDisconnect(self.name, self.outlet2.name, "C(OUT)")
        self.StreamConnect(self.name, self.outlet2.name, "H(OUT)")
        self.StreamConnect(self.name, self.outlet1.name, "C(OUT)")
        self.hotside_pres = new_hotside_pres
        self.coldside_pres = new_coldside_pres
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "PRES_HOT"
        ).Value = self.hotside_pres
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "PRES_COLD"
        ).Value = self.coldside_pres

    def undo_switch(self):
        # Switch the inlet streams back to their original state
        self.StreamDisconnect(self.name, self.inlet_stream2.name, "H(IN)")
        self.StreamDisconnect(self.name,self.inlet_stream1.name, "C(IN)")
        self.StreamConnect(self.name, self.inlet_stream1.name, "H(IN)")
        self.StreamConnect(self.name, self.inlet_stream2.name, "C(IN)")
        new_coldside_pres = self.hotside_pres
        new_hotside_pres = self.coldside_pres
        
        # Update the outlet streams accordingly
        self.StreamDisconnect(self.outlet2.name, "H(OUT)")
        self.StreamDisconnect(self.outlet1.name, "C(OUT)")
        self.StreamConnect(self.outlet1.name, "H(OUT)")
        self.StreamConnect(self.outlet2.name, "C(OUT)")
        self.hotside_pres = new_hotside_pres
        self.coldside_pres = new_coldside_pres
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "PRES_HOT"
        ).Value = self.hotside_pres
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "PRES_COLD"
        ).Value = self.coldside_pres

    def enery_consumption(self):
        q = abs(self.BLK.Elements(self.name).Elements("Output").Elements("HX_DUTY").Value)
        return q
    
class Pump(Block):
    def __init__(self, name, inlet_stream):
        super().__init__(name, "Pump")
        self.name = name
        self.press = 2.4  # Default pressure in bar
        self.inlet_stream = inlet_stream

        self.BlockCreate()

    def connect(self):
        # Inlet connection
        self.StreamConnect(self.name, self.inlet_stream.name, "F(IN)")
        self.BLK.Elements(self.name).Elements("Input").Elements("EFF").Value = 0.5
        self.BLK.Elements(self.name).Elements("Input").Elements("DEFF").Value = 0.9
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "OPT_SPEC"
        ).Value = "PRES"

        s = Stream(f"{self.name}OUT")
        self.StreamConnect(self.name, s.name, "P(OUT)")
        return s

    def dv_placement(self, press):
        self.BLK.Elements(self.name).Elements("Input").Elements("PRES").Value = press
    def enery_consumption(self):
        q = abs(self.BLK.Elements(self.name).Elements("Output").Elements("WNET").Value)
        return q

class CSTR_A(Block):
    def __init__(self, name, inlet_stream):
        super().__init__(name, "RCSTR")
        self.name = name
        self.inlet_stream = inlet_stream
        self.V = 30

        self.BlockCreate()

    def connect(self):
        # Inlet connection
        self.StreamConnect(self.name, self.inlet_stream.name, "F(IN)")

        # Reactors specifications
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "SPEC_OPT"
        ).Value = "DUTY"
        self.BLK.Elements(self.name).Elements("Input").Elements("DUTY").Value = 0
        self.BLK.Elements(self.name).Elements("Input").Elements("PHASE").Value = "L"

        # Reaction
        nodes = self.AspenSimulation.Application.Tree.FindNode(
            f"/Data/Blocks/{self.name}/Input/RXN_ID"
        ).Elements
        nodes.InsertRow(1, nodes.Count)
        nodes(nodes.Count - 1).Value = "EGR"
        s = Stream(f"{self.name}OUT")
        self.StreamConnect(self.name, s.name, "P(OUT)")
        return s
    
    def dv_placement(self, cstr_variables):
        self.BLK.Elements(self.name).Elements("Input").Elements("VOL").Value = cstr_variables[0]
        self.BLK.Elements(self.name).Elements("Input").Elements("PRES").Value = cstr_variables[1]

class PFR_A(Block):
    def __init__(self, name, inlet_stream):
        super().__init__(name, "RPlug")
        self.name = name
        self.inlet_stream = inlet_stream
        self.D = 0.5  # Default diameter in meters
        self.L = 10  # Default length in meters

        self.BlockCreate()

    def connect(self):
        # Inlet connection
        self.StreamConnect(self.name, self.inlet_stream.name, "F(IN)")

        # Reactors specifications
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "TYPE"
        ).Value = "ADIABATIC"

        # Sizing
        self.BLK.Elements(self.name).Elements("Input").Elements("NPHASE").Value = 1
        self.BLK.Elements(self.name).Elements("Input").Elements("PHASE").Value = "L"

        # Reaction
        nodes = self.AspenSimulation.Application.Tree.FindNode(
            f"/Data/Blocks/{self.name}/Input/RXN_ID"
        ).Elements
        nodes.InsertRow(1, nodes.Count)
        nodes(nodes.Count - 1).Value = "EGR"

        s = Stream(f"{self.name}OUT")
        self.StreamConnect(self.name, s.name, "P(OUT)")
        return s
    def dv_placement(self, dimensions):
        self.BLK.Elements(self.name).Elements("Input").Elements("LENGTH").Value = dimensions[0]
        self.BLK.Elements(self.name).Elements("Input").Elements("DIAM").Value = dimensions[1]
        # self.BLK.Elements(self.name).Elements("Input").Elements(
        #     "PRES"
        # ).Value = self.inlet_stream.get_press()   

class Column(Block):
    def __init__(self, name, inlet_stream):
        super().__init__(name, "Radfrac")
        self.name = name
        self.nstages = 12  # Default number of stages
        self.reflux_ratio = 0.024
        self.bottoms_flow = 26.3
        self.press = 2.4  # Default pressure in bar
        self.inlet_stream = inlet_stream

        self.BlockCreate()

    def connect(self):
        self.StreamConnect(self.name, self.inlet_stream.name, "F(IN)")

        # Configuration
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "CALC_MODE"
        ).Value = "EQUILIBRIUM"
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "NSTAGE"
        ).Value = self.nstages
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "CONDENSER"
        ).Value = "TOTAL"
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "REBOILER"
        ).Value = "KETTLE"
        self.BLK.Elements(self.name).Elements("Input").Elements("NO_PHASE").Value = 2
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "CONV_METH"
        ).Value = "STANDARD"

        # Streams
        self.BLK.Elements(self.name).Elements("Input").Elements("FEED_STAGE").Elements(
            self.inlet_stream.name
        ).Value = 7
        self.BLK.Elements(self.name).Elements("Input").Elements("FEED_CONVE2").Elements(
            self.inlet_stream.name
        ).Value = "ON-STAGE"

        # Convergence
        self.BLK.Elements(self.name).Elements("Input").Elements("MAXOL").Value = 200

        d = Stream(f"{self.name}DOUT")
        self.StreamConnect(self.name, d.name, "LD(OUT)")
        b = Stream(f"{self.name}BOUT")
        self.StreamConnect(self.name, b.name, "B(OUT)")
        return d, b
    
    def dv_placement(self, column_variables):
        self.BLK.Elements(self.name).Elements("Input").Elements("NSTAGE").Value = column_variables[0]
        self.BLK.Elements(self.name).Elements("Input").Elements("BASIS_RR").Value = column_variables[1]
        self.BLK.Elements(self.name).Elements("Input").Elements("BASIS_B").Value = column_variables[2]
        self.BLK.Elements(self.name).Elements("Input").Elements("PRES1").Value = column_variables[3]
    def enery_consumption(self):
        q1 = abs(
            self.BLK.Elements(self.name).Elements("Output").Elements("COND_DUTY").Value
        )
        q2 = abs(
            self.BLK.Elements(self.name).Elements("Output").Elements("REB_DUTY").Value
        )
        return q1 + q2

    def sizing(self):
        D = (
            self.BLK.Elements(self.name)
            .Elements("Subobjects")
            .Elements("Tray Sizing")
            .Elements("1")
            .Elements("Output")
            .Elements("DIAM4")
            .Elements("1")
            .Value
        )
        H = 1.2 * 0.61 * (self.nstages - 2)

        return D, H

class Compressor(Block):
    def __init__(self, name, inlet_stream):
        super().__init__(name, "COMPR")
        self.name = name
        self.inlet_stream = inlet_stream
        self.press = 2.4  # Default pressure in bar

        self.BlockCreate()

    def connect(self):
        # Inlet connection
        self.StreamConnect(self.name, self.inlet_stream.name, "F(IN)")
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "MODEL_TYPE"
        ).Value = "COMPRESSOR"
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "TYPE"
        ).Value = "ISENTROPIC"
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "OPT_SPEC"
        ).Value = "PRES"
        self.BLK.Elements(self.name).Elements("Input").Elements("NPHASE").Value = 2
        self.BLK.Elements(self.name).Elements("Input").Elements("SEFF").Value = 0.82

        s = Stream(f"{self.name}OUT")
        self.StreamConnect(self.name, s.name, "P(OUT)")
        return s
    
    def dv_placement(self, press):
        self.BLK.Elements(self.name).Elements("Input").Elements("PRES").Value = press
        
class Turbine(Block):
    def __init__(self, name, inlet_stream):
        super().__init__(name, "COMPR")
        self.name = name
        self.inlet_stream = inlet_stream
        self.press = 2.4  # Default pressure in bar

        self.BlockCreate()

    def connect(self):
        # Inlet connection
        self.StreamConnect(self.name, self.inlet_stream.name, "F(IN)")
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "MODEL_TYPE"
        ).Value = "TURBINE"
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "TYPE"
        ).Value = "ISENTROPIC"
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "OPT_SPEC"
        ).Value = "PRES"
        self.BLK.Elements(self.name).Elements("Input").Elements("NPHASE").Value = 2
        self.BLK.Elements(self.name).Elements("Input").Elements("SEFF").Value = 0.85

        s = Stream(f"{self.name}OUT")
        self.StreamConnect(self.name, s.name, "P(OUT)")
        return s

    def dv_placement(self, press):
        self.BLK.Elements(self.name).Elements("Input").Elements("PRES").Value = press

    def enery_consumption(self):
        q = abs(self.BLK.Elements(self.name).Elements("Output").Elements("WNET").Value)
        return q

class TriColumn(Block):
    def __init__(
        self, name, nstages, dist_rate, reflux_ratio, press, mid_rate, inlet_stream
    ):
        super().__init__(name, "Radfrac")
        self.name = name
        self.nstages = nstages
        self.dist_rate = dist_rate
        self.reflux_ratio = reflux_ratio
        self.press = press
        self.mid_rate = mid_rate
        self.inlet_stream = inlet_stream

        self.BlockCreate()

    def distill(self):
        self.StreamConnect(self.name, self.inlet_stream.name, "F(IN)")

        # Configuration
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "CALC_MODE"
        ).Value = "EQUILIBRIUM"
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "NSTAGE"
        ).Value = self.nstages
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "CONDENSER"
        ).Value = "TOTAL"
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "REBOILER"
        ).Value = "KETTLE"
        self.BLK.Elements(self.name).Elements("Input").Elements("NO_PHASE").Value = 2
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "CONV_METH"
        ).Value = "STANDARD"
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "BASIS_D"
        ).Value = self.dist_rate
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "BASIS_RR"
        ).Value = self.reflux_ratio

        # Streams
        self.BLK.Elements(self.name).Elements("Input").Elements("FEED_STAGE").Elements(
            self.inlet_stream.name
        ).Value = round(self.nstages / 3, 0)
        self.BLK.Elements(self.name).Elements("Input").Elements("FEED_CONVE2").Elements(
            self.inlet_stream.name
        ).Value = "ABOVE-STAGE"

        # Pressure
        self.BLK.Elements(self.name).Elements("Input").Elements(
            "PRES1"
        ).Value = self.press

        # Convergence
        self.BLK.Elements(self.name).Elements("Input").Elements("MAXOL").Value = 200

        # Tray sizing
        self.BLK.Elements(self.name).Elements("Subobjects").Elements(
            "Tray Sizing"
        ).Elements.Add("1")

        self.BLK.Elements(self.name).Elements("Subobjects").Elements(
            "Tray Sizing"
        ).Elements("1").Elements("Input").Elements("TS_STAGE1").Elements("1").Value = 2
        self.BLK.Elements(self.name).Elements("Subobjects").Elements(
            "Tray Sizing"
        ).Elements("1").Elements("Input").Elements("TS_STAGE2").Elements("1").Value = (
            self.nstages - 1
        )
        self.BLK.Elements(self.name).Elements("Subobjects").Elements(
            "Tray Sizing"
        ).Elements("1").Elements("Input").Elements("TS_TRAYTYPE").Elements(
            "1"
        ).Value = "SIEVE"

        d = Stream(f"{self.name}DOUT")
        self.StreamConnect(self.name, d.name, "LD(OUT)")
        mid = Stream(f"{self.name}MOUT")
        self.StreamConnect(self.name, mid.name, "SP(OUT)")
        b = Stream(f"{self.name}BOUT")
        self.StreamConnect(self.name, b.name, "B(OUT)")

        self.BLK.Elements(self.name).Elements("Input").Elements("PROD_PHASE").Elements(
            mid.name
        ).Value = "L"
        self.BLK.Elements(self.name).Elements("Input").Elements("PROD_STAGE").Elements(
            mid.name
        ).Value = round(self.nstages / 2, 0)
        self.BLK.Elements(self.name).Elements("Input").Elements("PROD_FLOW").Elements(
            mid.name
        ).Value = self.mid_rate

        return d, mid, b

    def enery_consumption(self):
        q1 = abs(
            self.BLK.Elements(self.name).Elements("Output").Elements("COND_DUTY").Value
        )
        q2 = abs(
            self.BLK.Elements(self.name).Elements("Output").Elements("REB_DUTY").Value
        )
        return q1 + q2

    def sizing(self):
        D = (
            self.BLK.Elements(self.name)
            .Elements("Subobjects")
            .Elements("Tray Sizing")
            .Elements("1")
            .Elements("Output")
            .Elements("DIAM4")
            .Elements("1")
            .Value
        )
        H = 1.2 * 0.61 * (self.nstages - 2)

        return D, H

class Empty_block(Block):
    def __init__(self, name):
        super().__init__(name, "Empty")
        self.name = name
    
    def dv_placement(self, *args):
        pass


In [164]:
class Flowsheet:
    def __init__(self, sim, inlet_specs):

        # Establish connection with ASPEN
        self.sim = sim
        # Declare the initial flowrate conditions
        self.inlet_specs = inlet_specs

        # Flowsheet
        self.info = {}
        self.unit_dict = {}
        self.stream_dict = {}
        self.mixer_count = 0
        self.splitter_count = 0
        self.heater_count = 0
        self.cooler_count = 0
        self.hex_count = 0
        self.pump_count = 0
        self.cstr_count = 0
        self.pfr_count = 0
        self.dc_count = 0
        self.compressor_count = 0
        self.turbine_count = 0
        self.subbranch_start = 0
        self.token17 =0
        self.token18 =0
        self.token19 =0
        self.token21 =0
        self.token22 =0
        self.reset()

    def get_outputs(self, sout):
        T = sout.get_temp()
        P = sout.get_press()
        Feo = sout.get_molar_flow("EO")
        Fw = sout.get_molar_flow("W")
        Feg = sout.get_molar_flow("EG")
        Fdeg = sout.get_molar_flow("DEG")
        out_list = [T, P, Feo, Fw, Feg, Fdeg]

        return out_list

    def Flowsheet_Building(self,equipment):
        """
        Putting the unit operations in ASPEN flowsheet without running the simulation and providing the decision variables.
        :param inlet_stream: The feed streams that will be used in the flowsheet.
        :param equipment: The numpy array of the unit operations that will be used in the flowsheet.
        """
        sin = self.reset()
        if len(sin) > 0:
            self.mixer_count +=1
            uo_name = f"M{self.mixer_count}"
            self.unit_dict[uo_name] = InitialMixer(uo_name, sin)
            sin = self.unit_dict[uo_name].connect()
        for i,uo in enumerate(equipment):
            if uo == 1:
                self.mixer_count += 1
                uo_name = f"M{self.mixer_count}"
                self.unit_dict[uo_name] = Mixer(uo_name, sin)
                sin = self.unit_dict[uo_name].connect(sin)
            elif uo == 2:
                self.splitter_count += 1
                self.subbranch_start += 1
                uo_name = f"S{self.splitter_count}"
                self.unit_dict[uo_name] = Splitter(uo_name,sin)
                sin,s2 = self.unit_dict[uo_name].connect()
                sb_name = f"SB{self.subbranch_start}"
                self.stream_dict[sb_name] = s2
                
            elif uo == 3:
                self.heater_count += 1
                uo_name = f"H{self.heater_count}"
                self.unit_dict[uo_name] = Heater(uo_name,sin)
                sin = self.unit_dict[uo_name].connect()
            elif uo == 4:
                self.cooler_count += 1
                uo_name = f"C{self.cooler_count}"
                self.unit_dict[uo_name] = Cooler(uo_name,sin)
                sin = self.unit_dict[uo_name].connect()
            elif uo == 5:
                pass
                #single token hex does not look feasible if there is more than one hex
                # self.hex_count += 1
                # uo_name = f"HX{self.hex_count}"
                # self.unit_dict[uo_name] = HeatExchanger(uo_name, sin)
            elif uo == 6:
                self.pump_count += 1
                uo_name = f"PP{self.pump_count}"
                self.unit_dict[uo_name] = Pump(uo_name,sin)
                sin = self.unit_dict[uo_name].connect()
            elif uo == 7:
                self.cstr_count += 1
                uo_name = f"CSTR{self.cstr_count}"
                self.unit_dict[uo_name] = CSTR_A(uo_name,sin)
                sin = self.unit_dict[uo_name].connect()
            elif uo == 8:
                self.pfr_count += 1
                uo_name = f"PFR{self.pfr_count}"
                self.unit_dict[uo_name] = PFR_A(uo_name,sin)
                sin = self.unit_dict[uo_name].connect()
            elif uo == 9:
                self.subbranch_start += 1
                self.dc_count += 1
                uo_name = f"DC{self.dc_count}"
                self.unit_dict[uo_name] = Column(uo_name,sin)
                d, b = self.unit_dict[uo_name].connect()
                sin = d
                sb_name = f"SB{self.subbranch_start}"
                self.stream_dict[sb_name] = b
            elif uo == 10:
                self.dc_count += 1
                self.subbranch_start += 1
                uo_name = f"DCR{self.dc_count}"
                self.unit_dict[uo_name] = Column(uo_name,sin)
                d,b = self.unit_dict[uo_name].connect()
                sin = b
                sb_name = f"SB{self.subbranch_start}"
                self.stream_dict[sb_name] = d
            elif uo == 11:
                self.compressor_count += 1
                uo_name = f"COMP{self.compressor_count}"
                self.unit_dict[uo_name] = Compressor(uo_name,sin)
                sin = self.unit_dict[uo_name].connect()
            elif uo == 12:
                self.turbine_count += 1
                uo_name = f"TURB{self.turbine_count}"
                self.unit_dict[uo_name] = Turbine(uo_name,sin)
                sin = self.unit_dict[uo_name].connect()
            elif uo == 13:
                # self.unit_dict["Product"] = sin
                self.unit_dict["Product"] = Empty_block("Product")
                #Main branch is finished with a product or a waste stream
                pass
            elif uo == 14:
                sin = self.stream_dict["SB1"]
                # self.unit_dict["sbs1"] = sin
                self.unit_dict["sbs1"] = Empty_block("sbs1")
            elif uo == 15:
                sin = self.stream_dict["SB2"]
                # self.unit_dict["sbs2"] = sin
                self.unit_dict["sbs2"] = Empty_block("sbs2")
            elif uo == 16:
                sin = self.stream_dict["SB3"]
                # self.unit_dict["sbs3"] = sin
                self.unit_dict["sbs3"] = Empty_block("sbs3")
            elif uo == 17:
                self.token17 += 1
                # self.unit_dict[f"SB1{self.token17}"] = sin
                self.unit_dict[f"SB1{self.token17}"] = Empty_block(f"SB1{self.token17}")
                if self.token17 == 1:
                    token17_connection_index = i+1
                if self.token17 == 2:
                    keys = list(self.unit_dict.keys())
                    uo_name = keys[token17_connection_index]
                    self.unit_dict[uo_name].connect(sin)
            elif uo == 18:
                self.token18 += 1
                # self.unit_dict[f"SB2{self.token18}"] = sin
                self.unit_dict[f"SB2{self.token18}"] = Empty_block(f"SB2{self.token18}")
                if self.token18 == 1:
                    token18_connection_index = i+1
                if self.token18 == 2:
                    keys = list(self.unit_dict.keys())
                    uo_name = keys[token18_connection_index]
                    self.unit_dict[uo_name].connect(sin)
            elif uo == 19:
                self.token19 += 1
                # self.unit_dict[f"SB3{self.token19}"] = sin
                self.unit_dict[f"SB3{self.token19}"] = Empty_block(f"SB3{self.token19}")
                if self.token19 == 1:
                    token19_connection_index = i+1
                if self.token19 == 2:
                    keys = list(self.unit_dict.keys())
                    uo_name = keys[token19_connection_index]
                    self.unit_dict[uo_name].connect(sin)
            elif uo == 20:
                pass
            elif uo == 21:
                self.token21 += 1
                if self.token21 == 1:
                    self.hex_count += 1
                    uo_name = f"HX{self.hex_count}"
                    self.unit_dict[uo_name] = HeatExchanger(uo_name, sin)
                    sin = self.unit_dict[uo_name].connect(self.token21)
                    hexa_index = i
                if self.token21 ==2:
                    keys = list(self.unit_dict.keys())
                    uo_name = keys[hexa_index]
                    sin = self.unit_dict[uo_name].connect(self.token21,sin2=sin)
                    nuo_name = f"{uo_name}{self.token21}"
                    self.unit_dict[nuo_name] = self.unit_dict[uo_name]
            elif uo == 22:
                self.token22 += 1
                if self.token22 == 1:
                    self.hex_count += 1
                    uo_name = f"HX{self.hex_count}"
                    self.unit_dict[uo_name] = HeatExchanger(uo_name, sin)
                    sin = self.unit_dict[uo_name].connect(self.token22)
                    hexb_index = i
                else:
                    keys = list(self.unit_dict.keys())
                    uo_name = keys[hexb_index]
                    sin = self.unit_dict[uo_name].connect(self.token22,sin2=sin)
                    nuo_name = f"{uo_name}{self.token22}"
                    self.unit_dict[nuo_name] = self.unit_dict[uo_name]

    def Flowsheet_Simulation(self, equipment,decision_variables):
        """
        Running the flowsheet simulation with the given decision variables using the ASPEN simulation environment.
        :param equipment: The numpy array of the unit operations that will be used in the flowsheet.
        :param decision_variables: The list of the decision variables that will be used in the flowsheet.
        """
        for eq, dv,uo in zip(equipment, decision_variables,self.unit_dict.values()):
            uo.dv_placement(dv)
            #eq = 0,1,13-20 no action
            # if eq ==2:
            #     #Splitter
            #     uo.dv_placement(dv)
            # elif eq == 3:
            #     #Heater
            #     uo.dv_placement(dv)
            # elif eq == 4:
            #     #Cooler
            #     uo.dv_placement(dv)
            # elif eq == 5:
            #     #Hex
            #     pass
            # elif eq == 6:
            #     #Pump
            #     uo.dv_placement(dv)
            # elif eq == 7:
            #     #CSTR
            #     uo.dv_placement(dv)
            #     # we need to check this
            # elif eq == 8:
            #     #PFR
            #     uo.dv_placement(dv)
            # elif eq == 9:
            #     #Column
            #     uo.dv_placement(dv)
            # elif eq == 10:
            #     #Column with recycle
            #     uo.dv_placement(dv)
            # elif eq == 11:
            #     #Compressor
            #     uo.dv_placement(dv)
            # elif eq == 12:
            #     #Turbine
            #     uo.dv_placement(dv)
            # elif eq == 21:
            #     #Heat Exchanger A
            #     uo.dv_placement(dv)
            # elif eq == 22:
            #     #Heat Exchanger B
            #     uo.dv_placement(dv)
        
    def render(self):
        for i in self.info:
            print(f"{i}: {self.info[i]}")

    def reset(self):
        # Reset all instances
        self.iter = 0
        self.sim.Reinitialize()
        # inlet_specs is going to be a list of lists which contains variables and dictionaries for compounds
        sin = [None] * len(self.inlet_specs)
        for i in range(len(self.inlet_specs)):
            sin[i] = Stream(f"IN{i+1}", self.inlet_specs[i])

        self.info = {}
        self.unit_dict = {}
        self.stream_dict = {}
        self.mixer_count = 0
        self.splitter_count = 0
        self.heater_count = 0
        self.cooler_count = 0
        self.hex_count = 0
        self.pump_count = 0
        self.cstr_count = 0
        self.pfr_count = 0
        self.dc_count = 0
        self.compressor_count = 0
        self.turbine_count = 0
        self.subbranch_start = 0
        self.token17 =0
        self.token18 =0
        self.token19 =0
        self.token21 =0
        self.token22 =0

        return sin


In [165]:
global sim
cwd = os.getcwd() # get current working directory
os.chdir(cwd) # change to the directory where the script is located
sim = Simulation("base.bkp",cwd,False) # create the base case of the simulation

Working Directory: c:\Users\m85830ak\Python\flowsheet-desing-main\EG
Aspen File: base.bkp


In [166]:
PFD_kwargs = {
        "sim": sim,
        "inlet_specs": [[298, 2.4, {"EO": 0.0, "W": 26.31, "EG": 0.0, "DEG": 0.0}],[298, 2.4, {"EO": 27.62, "W": 0.0, "EG": 0.0, "DEG": 0.0}],
                        [360.27,2.4,{"EO": 0.86483855131462, "W": 521.25960426966, "EG": 0.253500051247613, "DEG": 1.05869515878343E-07}],
        ]
}
# Between different design it requires to create a new flowsheet instance
PFD = Flowsheet(**PFD_kwargs) # create a flowsheet instance

In [167]:
#Nabil ED5
#THxa2MHxbCCompSHxb1MSH3MH-1H1-2HxaST2-33
equipment = np.array([0,12,21,18,1,22,4,11,2,22,17,1,2,3,19,1,3,14,3,17,15,21,2,12,18,16,19,20])
PFD.Flowsheet_Building(equipment)

In [168]:
decision_variables=[
    0, # Initial Mixer
    79, #Turbine Discharge Pressure
    10, #Heat Exchanger A Approach Temperature
    0, #Subbranch 2 mixer connector
    0, #Mixer 1
    4, #Heat Exchanger B Approach Temperature
    35, #Cooler Temperature
    299, #Compressor Discharge Pressure
    0.95, #Splitter 1 Split Fraction
    4, #Heat Exchanger B Approach Temperature
    0, #Subbranch 1 mixer connector
    0, #Mixer 2
    0.75, #Splitter 2 Split Fraction
    180, #Heater Temperature
    0, #Subbranch 3 end
    0, #Mixer 3
    280, #Heater Temperature
    0, #Subbranch 1 start
    150 , #Heater Temperature
    0, #Subbranch 1 end
    0, #Subbranch 2 start
    10, #Heat Exchanger A Approach Temperature
    0.55, #Splitter 3 Split Fraction
    78, #Turbine Discharge Pressure
    0, #Subbranch 2 end
    0, #Subbranch 3 start
    0, #Subbranch 3 end
    ]
PFD.Flowsheet_Simulation(equipment,decision_variables)
# PFD.unit_dict

In [169]:
PFD = Flowsheet(**PFD_kwargs) # create a flowsheet instance
equipment = np.array([0,17,1,3,8,4,10,13,14,6,17,20]) # PFR design
PFD.Flowsheet_Building(equipment)

In [170]:
decision_variables = [0,0,0,355,[10,3],398,[12,0.024,26.3,0.71],0,0,2.4,0,0] # PFR design
PFD.Flowsheet_Simulation(equipment,decision_variables)

In [171]:
# env = Flowsheet(**env_kwargs) # create a flowsheet instance
equipment = np.array([0,17,1,3,7,7,7,7,7,7,7,7,4,10,13,14,6,17,20]) # CSTR design
PFD.Flowsheet_Building(equipment)

In [172]:
decision_variables = [0,0,0,355,[3.75,2.4],[3.75,2.4],[3.75,2.4],[3.75,2.4],[3.75,2.4],[3.75,2.4],[3.75,2.4],[3.75,2.4],398,[12,0.024,26.3,0.71],0,0,2.4,0,0] # CSTR design
PFD.Flowsheet_Simulation(equipment,decision_variables)

In [173]:
PFD.sim.EngineReinit() # initialize the simulation engine
PFD.sim.EngineRun() # run the simulation

In [ ]:
for key in ["PER_ERROR", "PER_WARNING", "PER_MESSAGE"]:
    node = PFD.sim.AspenSimulation.Tree.FindNode(f"/Data/Results Summary/Run-Status/Output/{key}")
    if node and node.Value:
        for child in node.Elements:
            print(f"{child.Name}: {child.Value}")

In [175]:
PFD.sim.Quit() # close the simulation